## 1.Introduction to PyTorch, a Deep Learning Library

### 1-1. Tensors
- Similar to array or matrix
- Building block of neural networks 
##
- Can be added or substracted, provided that their shapes are compatible
- a * b : element-wise multiplication vs a @ b = Matrix multiplication <br>

        → Perform addition and multiplication to process data and learn patterns 

In [25]:
# Import torch
import torch

temperatures = [[72, 75, 78], [70, 73, 76]] # can be created from Python lists or Numpy arrays

# Create a tensor from temperatures
temperatures= torch.tensor(temperatures)

print(temperatures)


tensor([[72, 75, 78],
        [70, 73, 76]])


### 1-2. Cheking and adding tensors

In [2]:
adjustment = torch.tensor([[2, 2, 2], [2, 2, 2]])

# Display the shape of the adjustment tensor
print("Adjustment shape:", adjustment.shape)

# Display the type of the adjustment tensor

print("Adjustment type:", adjustment.dtype)

print("Temperatures shape:", temperatures.shape)
print("Temperatures type:", temperatures.dtype)


Adjustment shape: torch.Size([2, 3])
Adjustment type: torch.int64
Temperatures shape: torch.Size([2, 3])
Temperatures type: torch.int64


In [3]:
adjustment = torch.tensor([[2, 2, 2], [2, 2, 2]])

# Add the temperatures and adjustment tensors
corrected_temperatures = adjustment + temperatures
print("Corrected temperatures:", corrected_temperatures)


Corrected temperatures: tensor([[74, 77, 80],
        [72, 75, 78]])


### 1-3. Neural networks and layers
- A neural network consists of input(dataset features), hidden, and output layers(predictions).
- ''Fully connected network''(= linear model) 
    - Every input neuron connects to every output neuron 
    - Network with no hidden layers where the output layer is a linear layer
    - Linear layer : input @ wieghts + bias = output


- weight : Reflects the importance of different features
- bias : Provides the neuron with a baseline output 

ex) 
- input (temperature, humidity, wind) → output (rain, cloudy) <br>
humidity feature will have a more significant weight<br>
bias is to account for baseline information


In [4]:
# Linear layer network

import torch
import torch.nn as nn

input_tensor = torch.tensor([[0.3471, 0.4547, -0.2356]])

# Create a Linear layer 
linear_layer = nn.Linear (
    in_features= 3,
    out_features= 2
)

# Pass input_tensor through the linear layer 
output = linear_layer(input_tensor)

print(output)

tensor([[0.6501, 0.5737]], grad_fn=<AddmmBackward0>)


### 1-4. Hidden layer and parameters
- Stacking layes with nn.Seqential()

Layer are made of neurons 
- Fully connected : each neuron links to all neurons in the previous layer 
- A neuron in a linear layer :
    - perfroms a linear operation using all neurons from the previous layer 
    - Has N+1 parameters : N from inputs and 1 for the bias

In [5]:
import torch 
import torch.nn as nn

input_tensor = torch.Tensor([[2, 3, 6, 7, 9, 3, 2, 1]])

# Create a container for stacking linear layers
model = nn.Sequential(
    nn.Linear(8,4),
    nn.Linear(4,1)
) # cf) nn.Linear(input features, output features)

output = model(input_tensor)
print(output)


tensor([[-4.0877]], grad_fn=<AddmmBackward0>)


In [ ]:
# Counting the number of parameters

model = nn.Sequential(nn.Linear(9, 4),  # 9 * 4 / 1 * 4
                      nn.Linear(4, 2),  # 2 * 4 / 1 * 2 
                      nn.Linear(2, 1))  # 1 * 2 / 1 * 1

total = 0
for parameter in model.parameters():
    print(parameter)  # Weight & bias
    total += parameter.numel()
    print(total)
print(total)


Parameter containing:
tensor([[ 0.2891,  0.1186,  0.0401, -0.0963, -0.1844, -0.0411, -0.1135,  0.2212,
          0.2396],
        [-0.1951,  0.1415, -0.2942,  0.0370,  0.1870, -0.0499, -0.2725, -0.0245,
          0.1293],
        [-0.2077,  0.0454, -0.1874,  0.2431, -0.0677, -0.0499,  0.3330,  0.3204,
         -0.0576],
        [ 0.2808,  0.3320,  0.0691, -0.0800,  0.0241, -0.0213,  0.2417,  0.1148,
         -0.0924]], requires_grad=True)
36
Parameter containing:
tensor([-0.2468,  0.1031, -0.1562,  0.0858], requires_grad=True)
40
Parameter containing:
tensor([[ 0.0347,  0.0269, -0.0122, -0.2738],
        [-0.0014, -0.1378, -0.3543, -0.4095]], requires_grad=True)
48
Parameter containing:
tensor([-0.0818, -0.4063], requires_grad=True)
50
Parameter containing:
tensor([[-0.0614, -0.4127]], requires_grad=True)
52
Parameter containing:
tensor([-0.2353], requires_grad=True)
53
53


----

## Week 1 review

###  Section 1: Implementing a simple linear NN

In [7]:
class Net(nn.Module):
    def __init__(self) : 
        super(Net, self).__init__()  # Initalise parent class
        self.fc1 = nn.Linear(2,1) # Output layer 

    def forward(self, x):  # The function we specify which function to apply on the input 
        x = self.fc1(x)   # 입력값에 가중치 곱하고 편향 더해서 계산하는 것 
        return x

In [8]:
net1 = Net()

list(net1.parameters())

# requires_grad=True : 자동 미분 속성 
# → Backpropagation 알고리즘을 실행할 경우, Pytorch가 이 매개변수들을 최적화(학습)할 것임을 의미한다
# → 매개변수를 최적화 하려면 손실 값 (loss value)에 따른 기울기를 저장해야하는데 이 때 이 키워드가 필요한다

[Parameter containing:
 tensor([[-0.4788,  0.2652]], requires_grad=True),
 Parameter containing:
 tensor([0.4565], requires_grad=True)]

In [9]:
# testing whether nerons works
x_input = torch.tensor([[0.1, 0.5]])
net1(x_input)

tensor([[0.5412]], grad_fn=<AddmmBackward0>)

In [10]:
# (0.1* - 0.6601) + (0.5 * -0.0942) + -0.2797

### Section 2: Implementing a perceptron in PyTorch
- 퍼셉트론 : 중간층(은닉층이 없음!) 오직 입력층과 아웃풋층 (step function을 activation function으로 사용함)
- 이렇게 만든 퍼셉트론은 "선형 분리" 문제만 풀 수 있습니다. (예: 앤드(AND) 게이트, 오어(OR) 게이트)
하지만 이 퍼셉트론 하나만으로는 절대 못 푸는 문제가 하나 있는데, 바로 유명한 XOR 문제입니다. 이 문제를 풀려면 아까 말씀하신 **'중간층(Hidden Layer)'**을 추가해야 하죠.

In [11]:
torch.Tensor(1)

tensor([0.])

In [ ]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(2,1) # Sum of input : 그 가중치랑 계산하는거 알지알지?
    
    def forward(self, x):
        x = self.fc1(x) 
        # 1 if x > 0 else 0
        # 활성화 여부를 결정함 (최종 결정의 스위치 역할 )
        # 두번째 인자 values는 입력이 정확이 0 일때 어떤 값을 줄것인가를 계산함 (x>0, x<0 만 정의해서 x= 0일때 정의해줘야함 여기는 1로 정의)
        # 요즘은 안씀. 미분이 불가해서 대신 가능한 sigmoid나 Lelu를 쓰는 것
        x = torch.heaviside(x, torch.Tensor(1))
        return x 

In [12]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        
        # Gap 1: single linear unit
        self.fc1 = nn.Linear(2, 1)
        
        # Hardcoded weights and bias
        with torch.no_grad(): # Pytorch는 기본적으로 모든 연산을 기록하기 때문에 no_grad = 계산 기록하지 마라 (그래프 생성 X) → backward 불가능 / ① 가중치 초기화 (지금 경우) ② 모델 평가 
            self.fc1.weight = nn.Parameter(torch.tensor([[1.0, 1.0]]))
            self.fc1.bias   = nn.Parameter(torch.tensor([-1.5]))
        
    def forward(self, x):
        x = self.fc1(x)
        
        # Gap 2: Heaviside step function
        x = (x > 0).float()
        return x



In [13]:
my_perceptron = Perceptron()
my_perceptron.fc1.weight.data = torch.tensor([[ 0.4, 0.2]])
my_perceptron.fc1.bias.data = torch.tensor([-0.3])

In [14]:
my_perceptron.fc1.bias

Parameter containing:
tensor([-0.3000], requires_grad=True)

In [24]:
test_inputs = torch.tensor([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0]
])

outputs = my_perceptron(test_inputs)

for x, y in zip(test_inputs, outputs):
    #print(f"Input {x} -> Output {int(y)}") #tensor value (y는 또같은데...)
    print(f"Input {x.tolist()} -> Output {int(y.item())}")




Input [0.0, 0.0] -> Output 0
Input [0.0, 1.0] -> Output 0
Input [1.0, 0.0] -> Output 1
Input [1.0, 1.0] -> Output 1


## Section 3: Solving a classification problem using a Perceptron

In [16]:
class ExNet(nn.Module):
    def __init__(self):
        super(ExNet, self).__init__()
        self.fc1 = nn.Linear(2,1)
        
        self.fc1.weights = torch.nn.Parameter(torch.tensor([[0.66],[1]])) # 둘다 그림에서 계산한거 
        self.fc1.bias = torch.nn.Parameter(torch.tensor([[-0.5]]))  
        
        self.heaviside = torch.heaviside
        
        
    def forward(self, x):
        x = self.fc1(x)
        output = self.heavside(x, 0.5)
        return output

## Section 4: Training a perceptron
- implement a method that trains the modelsuing some training data + test

In [17]:
data = [(0.7,0.3,1),(0.4,0.5,1), (0.6,0.9, 1), (0.2,0.2, 0), (0.1, 0.1, 0)]

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2,1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.heaviside(x, torch.Tensor(1))
        return x


In [18]:
training_data = torch.Tensor(
    [(0.7,0.3, 1),
     (0.4,0.5, 1),
     (0.6,0.9, 1),
     (0.2,0.2, 0)]
)
learning_rate = 0.1

# Initialise bias, weight1, weight2 
initial_weights = torch.Tensor( (-0.5, 0.3, -0.2) )

In [ ]:
# True : if the network fails to classify every sample in the data correctly.
# Whenever it returns True, it will imply that our network requires more training.
def keep_training(nn, data):
    for sample in data:
        if not torch.eq(nn(sample[0:2]), sample[-1]) : #input values 2, answer 1 
            # 모델 답하고 실제 답 비교해서 하나라도 다른게 있다면  
            return True #계속 하세요
    # # 6. 모든 문제를 다 돌았는데 틀린 게 하나도 없다면?
    return False

In [20]:
# write a function to implement the learing algorithm of page 24 
def train_perceptron(learning_rate, initial_weights, data):
    perceptron = Net()# Gap 1
    perceptron.fc1.bias.data = initial_weights[0]
    perceptron.fc1.weight.data = initial_weights[1:]
    
    
    while keep_training(perceptron, data):
        for sample in data:
            temp_output = perceptron(sample[0:2])
            label = sample[-1]
            delta_w = learning_rate * (label - temp_output) * torch.Tensor( [1, sample[0], sample[1]] )
            print(delta_w)
            perceptron.fc1.bias.data = perceptron.fc1.bias.data + delta_w[0]
            perceptron.fc1.weight.data = perceptron.fc1.weight.data + delta_w[1:]
            
    return perceptron     

In [ ]:
def train_perceptron(learning_rate, initial_weights, data):
    perceptron = Net()                     # Gap 1
    
    # initial_weights = (bias, w1, w2)
    perceptron.fc1.bias.data = initial_weights[0:1]           # Gap 2
    perceptron.fc1.weight.data = initial_weights[1:].view(1,2)  # ⭐️ Gap 3
    
    while keep_training(perceptron, data):
        for sample in data:
            x = sample[0:2]
            label = sample[-1]
            
            temp_output = perceptron(x)                         # Gap 4
            
            error = label - temp_output
            delta_w = learning_rate * error * x                 # Gap 5 (직관적으로 이해하면 원래 x 에서 learning_rate * loss ) 업데이트 !
            print(delta_w) 
            
            perceptron.fc1.bias.data += learning_rate * error   # Gap 6
            perceptron.fc1.weight.data += delta_w.view(1,2)     # ⭐️ Gap 7 (1행 2열 벡터로 만듬)
            
    return perceptron # We return the perceptron so we can use the trained model.

In [22]:
# Let us now run our algorithm, and see how far it goes in solving or classification problem!

train_perceptron(0.1, initial_weights, training_data)

tensor([0.0700, 0.0300], grad_fn=<MulBackward0>)
tensor([0.0400, 0.0500], grad_fn=<MulBackward0>)
tensor([0.0600, 0.0900], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([0.0400, 0.0500], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([-0.0200, -0.0200], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([0.0400, 0.0500], grad_fn=<MulBackward0>)
tensor([0., 0.], grad_fn=<MulBackward0>)
tensor([-0.0200, -0.0200], grad_fn=<MulBackward0>)


Net(
  (fc1): Linear(in_features=2, out_features=1, bias=True)
)

**Question:** *Compute the generalisation error of the perceptron trained above w.r.t. the ideal classification given in the above figure, i.e.  draw the classification boundary for the trained perceptron and compare it with that of the figure.*